In [ ]:
from requests.auth import HTTPDigestAuth
from pygerrit2 import GerritRestAPI, HTTPBasicAuth, HTTPBasicAuthFromNetrc, Anonymous
from IPython.display import clear_output
import requests
import pprint
import appscript
import numpy as np 
import matplotlib.pyplot as plt 
import urllib
import json;
import pandas as pd
import configparser
from gerrit import GerritClient
import csv
import os
import glob
import hashlib

In [ ]:
auth = Anonymous()
gerrit_url= 'https://gerrit-review.googlesource.com'
rest = GerritRestAPI(url=gerrit_url, auth = auth)
projects = rest.get("/projects/?d")

# Get projects accessible by the caller. 
# A map is returned that maps the project names to ProjectInfo entries. 
# The entries in the map are sorted by project
# As result a map is returned that maps the project names to ProjectInfo entries

for key in projects: 
    value = projects[key]
    identifier = value.get("id")
    name = value.get("web_links")[0:1]
    
    changes = get_changes_in_project(name)
    code_changes= get_code_changes()        

In [ ]:
# getting all changes in project
def get_changes_in_project(name):
    changes_in_project = rest.get("changes/?q=project:" + name + "&o=ALL_REVISIONS&o=ALL_COMMITS&o=ALL_FILES&o=MESSAGES&o=DETAILED_LABELS" , 
                                  headers={'Content-Type': 'application/json'})

    for i in range(len(changes_in_project)):
            revision = changes_in_project[i]['current_revision']
            change_id = changes_in_project[i]['change_id']
            get_review_comments(change_id, revision)

In [ ]:
# getting all review comments
def get_review_comments(id, revision):
        review_count = 0
        
        reviews_in_revisions = rest.get("/changes/" + str(id)+ "/revisions/" + str(revision)+ "/comments/", 
                                        headers={'Content-Type': 'application/json'})
        
        for index_comment in range(len(reviews_in_revisions[list(reviews_in_revisions.keys())[0]])):
            review_count = review_count + 1
            filename = str(next(iter(reviews_in_revisions)))
            
            if 'id' in reviews_in_revisions[list(reviews_in_revisions.keys())[0]][index_comment]:
                comment_id = reviews_in_revisions[list(reviews_in_revisions.keys())[0]][index_comment]['id']
            else:
                comment_id = 'not found'
                
            if 'message' in reviews_in_revisions[list(reviews_in_revisions.keys())[0]][index_comment]:
                review_comment = reviews_in_revisions[list(reviews_in_revisions.keys())[0]][index_comment]['message']
            else:
                review_count = 'not found'
                
            if 'line' in reviews_in_revisions[list(reviews_in_revisions.keys())[0]][index_comment]:
                commented_lines = reviews_in_revisions[list(reviews_in_revisions.keys())[0]][index_comment]['line']
            else:
                commented_lines = 'not found'

            if 'range' in reviews_in_revisions[list(reviews_in_revisions.keys())[0]][index_comment]:
                comment_range = reviews_in_revisions[list(reviews_in_revisions.keys())[0]][index_comment]['range']
            else:
                comment_range = 'not found'

            if '_account_id' in reviews_in_revisions[list(reviews_in_revisions.keys())[0]][index_comment]:
                account_id = reviews_in_revisions[list(reviews_in_revisions.keys())[0]][index_comment]['_account_id']
            else:
                account_id = 'not found'

            if 'commit_id' in reviews_in_revisions[list(reviews_in_revisions.keys())[0]][index_comment]:
                commit_id = reviews_in_revisions[list(reviews_in_revisions.keys())[0]][index_comment]['commit_id']
            else:
                commit_id = 'not found'

            with open('gerrit_review_comments_revision.csv', 'a', newline='') as file:
                            writer = csv.writer(file, delimiter='$')
                            writer.writerow([str(row["project"]), str(row["branch"]),str(row["Id"]),
                                             str(row["revision"]), str(row["change_id"]), filename, 
                                             comment_id, review_comment, commented_lines,
                                             comment_range, account_id, commit_id,
                                             str(row["creation_date"])])

        review_count = review_count + 1

In [ ]:
def get_code_changes():
    review_comments_ = pd.read_csv(r'gerrit_review_comments_revision.csv', sep='$')
    
    for index, row in review_comments_.iterrows():
        try:
            reviewed_comments = rest.get("/changes/" + str(row['Id']) + 
                                         "/revisions/" + str(row["revision"])+ 
                                         "/files/" + str(row["filename"]) + 
                                         "/diff", 
                                         headers={'Content-Type': 'application/json'})

            content = reviewed_comments['content']
            for key in range(len(content)):
                    if next(iter(content[key])) == 'b': # needed to retrive changes to files, not deletion
                        loc = list(next(iter(content[key].values())))
                    else:
                        continue

                    for line in loc:
                            with open('gerrit_comments_code_revision_all_states.csv', 'a', newline='') as file:
                                        writer = csv.writer(file, delimiter='$')
                                        writer.writerow([str(row["project"]), str(row["branch"]),
                                                         str(row["Id"]),str(row["revision"]), 
                                                         str(row["change_id"]), str(row["filename"]),
                                                         str(row["comment_id"]),
                                                         str(row["review_comment"]), 
                                                         str(row["commented_lines"]),
                                                         str(row["comment_range"]), 
                                                         str(row["account_id"]), 
                                                         str(row["commit_id"]),
                                                         str(row["creation_date"]), 
                                                         line])

                    else:
                        continue
        except (Exception, IndexError):
            reviewed_comments = 'null'  